In [ ]:
%load_ext tensorboard

In [ ]:
!touch $HOME/.cdsapirc
!echo "url: https://cds.climate.copernicus.eu/api/v2" >> $HOME/.cdsapirc
!echo "key: 71023:cf0744cc-00c0-47d2-b625-926642aa75e0" >> $HOME/.cdsapirc

In [ ]:
!pip install 

In [ ]:
!pip install cdsapi
# !pip install cfgrib
!apt install libeccodes-tools
!pip install missingno

In [ ]:
!ls

# Imports

In [ ]:
import cdsapi
import xarray as xr
import matplotlib.pyplot as plt
import missingno as msno 
import numpy as np
import scipy.stats as stats
import pandas as pd

import torch
from torch import nn

import os
import json
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch.optim as optim
import torch
from torch import nn
from torch.nn import functional as F
import pdb
import tqdm
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter()

### We have to use daskt to handle such big dataset
from dask.distributed import Client

client = Client()
client

In [ ]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

# Download Data

Let's download the data from the CDS API,

Here I am only using 3 days of data because interpolating with anything bigger than it is exhausting my 12GB Ram limit on "collab". And "dask" does not support Interpolation.

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'netcdf',
        'variable': [
            '2m_temperature', 'total_precipitation', 'volumetric_soil_water_layer_1',
        ],
        'year': '2019',
        'month': '12',
        'time': '12:00',
        'day': [
              '01', '02','03',
              # '04', '05', '06',
              # '07', '08', '09',
              # '10', '11', '12',
              # '13', '14', '15',
        ],
    },
    'download.nc')


In [ ]:
!ls

In [ ]:
dat = xr.open_dataset("download.nc",
    # chunks={
    #     "latitude": 250,
    #     "longitude": 250,
    #     "time": -1,
    # },  # this tells xarray to open the dataset as a dask array
)
with xr.set_options(display_style="text"):
  display(dat)

# Read DATA

We are doing some extra calculations from getting a general Idea for our dataset.

In [ ]:
with xr.set_options(display_style="text"):
  lon,lat,t = dat.indexes.values()
  print(f"Lon Range: {lon[0]}, {lon[-1]}")
  print(f"Lat Range: {lat[0]}, {lat[-1]}")
  print(f"Time Range: {t[0]}, {t[-1]}")
  print(f"Data values i.e t2m \n min:{dat['t2m'].min().values}, max:{dat['t2m'].max().values}, count:{dat['t2m'].count().values}")
  print(f"Data values i.e tp \n min:{dat['tp'].min().values}, max:{dat['tp'].max().values}, count:{dat['tp'].count().values}")
  print(f"Data values i.e swvl1 \n min:{dat['swvl1'].min().values}, max:{dat['swvl1'].max().values}, count:{dat['swvl1'].count().values}")

## Till now we know
~ It is a 3d data.

~ Axis beign 
  * longitude  (longitude)  0.0, 359.8999938964844
  * latitude   (latitude) 90.0, -90.0
  * time       (time) 2019-12-01 12:00:00, 2019-12-15 12:00:00

~ Then for each of our datasets we have our data values, ranging between.
* Temperature goes from222.674560546875 to 315.7596740722656
-> temperature goes from -15 to 41.85 C. Seems reasonable.

* total_precipitation i.e t2m min:7.450580596923828e-09,max: 0.21889278292655945
-> Such low values are reasonable as in most places it doesn't rain. But It is in meters let's convert it into millimeters

* volumetric_soil_water_layer_1 i.e 
min:0.0, max: 0.7660064697265625
-> It's valued is in m^3 m^(-3). So I don't think we should convert it. Because it is water per unit of volume.


## EDA

##  mean, median, standard dev., variance, range, spatial resolution

# Data Values from all the data.

In [ ]:
print(
    "\tt2\t\ttp\t\tswvl1\n"
)
print("Mean\t",dat.mean(axis=(0,1,2)).to_array().values)
print("Median\t",dat.median(axis=(0,1,2)).to_array().values)
print("Std\t",dat.std(axis=(0,1,2)).to_array().values)
print("Var\t",dat.var(axis=(0,1,2)).to_array().values)

Here we are calculating these values for the entirety of our data set. 
Herewith mean we are trying to see where our data lies within each sub-domain["t2", ... etc]. So the temperature lies somewhere around 268 k and similarly precipitation mean is in 1e-4 and volumetric water is in 1e-1. We might need to normalize our data to make it easier for our neural network to learn.

## Data Values on each data, i.e. averaging over latitude and longtitude.

In [ ]:
dat["t2m"]

In [ ]:
fig, axes = plt.subplots(3, 3)
fig.set_figheight(15)
fig.set_figwidth(20)

## MEAN
axes[0][0].plot(dat["t2m"].mean(("latitude", "longitude")))
axes[0][0].set_title(label="2m_temperature_mean")
axes[1][0].plot(dat["swvl1"].mean(("latitude", "longitude")))
axes[1][0].set_title(label="volumetric_soil_water_layer_1_mean")
axes[2][0].plot(dat["tp"].mean(("latitude", "longitude")))
axes[2][0].set_title(label="total_precipitation_mean")

# MEDIAN
axes[0][1].plot(dat["t2m"].median(("latitude", "longitude")))
axes[0][1].set_title(label="2m_temperature_median")
axes[1][1].plot(dat["swvl1"].median(("latitude", "longitude")))
axes[1][1].set_title(label="volumetric_soil_water_layer_1_median")
axes[2][1].plot(dat["tp"].median(("latitude", "longitude")))
axes[2][1].set_title(label="total_precipitation_median")

#STD
axes[0][2].plot(dat["t2m"].std(("latitude", "longitude")))
axes[0][2].set_title(label="2m_temperature_std")
axes[1][2].plot(dat["swvl1"].std(("latitude", "longitude")))
axes[1][2].set_title(label="volumetric_soil_water_layer_1_std")
axes[2][2].plot(dat["tp"].std(("latitude", "longitude")))
axes[2][2].set_title(label="total_precipitation_std")

plt.legend()

Here we can observer direct co-relation between tempreature,volumetric water and total precipitation.
I am trying to mean over all of latitude and longitude to get a mean on all days of our dataset. 
Lower temprature give us more precipitaton, but sometime heigher temprature get's more humidity so the volumetric soil water is heigher which is directly proportional to precipitaton. 

Plot a randomly chosen day from each of the datasets and use a sequential colormap for the plot. 


In [ ]:
fig, axes = plt.subplots(3)
fig.set_figheight(15)
fig.set_figwidth(20)
dat["t2m"].sel(time="2019-12-01").plot(ax=axes[0])
axes[0].set_title(label="2m_temperature_mean")
dat["swvl1"].sel(time="2019-12-01").plot(ax=axes[1])
axes[1].set_title(label="volumetric_soil_water_layer_1_mean")
dat["tp"].sel(time="2019-12-01").plot(ax=axes[2]) ## Having trouble because of highly skewed data.
axes[2].set_title(label="total_precipitation_mean")
plt.show()

Places like Africa have more temperature and less volumetric water and vice-versa. But our total precipitation is awfully blank. Let's investigate

In [ ]:
fig, axes = plt.subplots(3)

dat["t2m"].plot(ax=axes[0]) ## Having trouble because of highly skewed data.
axes[0].set_title(label="2m_temperature")

dat["swvl1"].plot(ax=axes[1]) ## Having trouble because of highly skewed data.
axes[1].set_title(label="volumetric_soil_water_layer_1")

dat["tp"].plot(ax=axes[2]) ## Having trouble because of highly skewed data.
axes[2].set_title(label="total_precipitation")

plt.show()

Here our problem is much more clear. We have left-skewed data in total precipitation. It's is expected though, since most places do not have that much precipitation and few places will have much more precipitation. We need to do change this data to make it easy for our neural network and not to get stuck in local minima of 0.0 value.

## Solving Skewd Total Precipitation


To solve left-skewed data we can do many things. 

The main one that works are. 
* Taking the log of every value in the dataset
* Taking a box cox of the dataset from scipy.stats package

Other methods include using min-max reduction, cube root, etc.

In [ ]:
box_cox_values = stats.boxcox(dat["tp"].values.ravel())

In [ ]:
plt.hist(box_cox_values, bins=12)

In [ ]:
plt.hist(dat["tp"].values.ravel(), bins=12)

In [ ]:
plt.hist(np.log(dat["tp"].values.ravel()), bins=12)

In [ ]:
dat["tp"] = np.log(dat["tp"])

Since taking a log every value is much faster than boxcox. I am going to use np.log. Also logging give us much more varied and spread data.

## Change Spatial Resolution

In [ ]:
dat = dat.interp(
    longitude=np.arange(90, -90, -0.05),
    latitude=np.arange(0, 360, 0.05),
)
# Lon Range: 0.0, 359.8999938964844
# Lat Range: 90.0, -90.0
# Time Range: 2019-12-01 12:00:00, 2019-12-15 12:00:00


In [ ]:
dat

In [ ]:
fig, axes = plt.subplots(3)

dat["t2m"].plot(ax=axes[0]) ## Having trouble because of highly skewed data.
axes[0].set_title(label="2m_temperature_mean")

dat["swvl1"].plot(ax=axes[1]) ## Having trouble because of highly skewed data.
axes[1].set_title(label="volumetric_soil_water_layer_1_mean")

dat["tp"].plot(ax=axes[2]) ## Having trouble because of highly skewed data.
axes[2].set_title(label="total_precipitation_mean")

plt.show()

In [ ]:
fig, axes = plt.subplots(3, 3)
fig.set_figheight(15)
fig.set_figwidth(20)

## MEAN
axes[0][0].plot(dat["t2m"].mean(("latitude", "longitude")))
axes[0][0].set_title(label="2m_temperature_mean")
axes[1][0].plot(dat["swvl1"].mean(("latitude", "longitude")))
axes[1][0].set_title(label="volumetric_soil_water_layer_1_mean")
axes[2][0].plot(dat["tp"].mean(("latitude", "longitude")))
axes[2][0].set_title(label="total_precipitation_mean")

# MEDIAN
axes[0][1].plot(dat["t2m"].median(("latitude", "longitude")))
axes[0][1].set_title(label="2m_temperature_median")
axes[1][1].plot(dat["swvl1"].median(("latitude", "longitude")))
axes[1][1].set_title(label="volumetric_soil_water_layer_1_median")
axes[2][1].plot(dat["tp"].median(("latitude", "longitude")))
axes[2][1].set_title(label="total_precipitation_median")

#STD
axes[0][2].plot(dat["t2m"].std(("latitude", "longitude")))
axes[0][2].set_title(label="2m_temperature_std")
axes[1][2].plot(dat["swvl1"].std(("latitude", "longitude")))
axes[1][2].set_title(label="volumetric_soil_water_layer_1_std")
axes[2][2].plot(dat["tp"].std(("latitude", "longitude")))
axes[2][2].set_title(label="total_precipitation_std")

plt.legend()

## Handle NAN

Some more things to do here

Since we are trying to predict precipitaion values as our task, so nan values for precipitaion is does not give use any information and changing them with mean values is also useless.
So we drop all values that have total precipitaton as nan and change for another.

In [ ]:
dat["tp"].sel(time="2019-12-01").isnull().sum(dim="latitude")

In [ ]:
dat["tp"].sel(time="2019-12-01").isnull().sum(dim="longitude").plot()

In [ ]:
dat["tp"].sel(time="2019-12-01").isnull().sum(dim="latitude").plot()

For some values of latitude we have almost all nan values make sence since the values in pacific ocean or north pole is not going to be available.

In [ ]:
dat["t2m"].isnull()

rn removing all nan values for clearity... and converting them to dataframe

In [ ]:
dat = dat.to_dataframe()

In [ ]:
dat.dropna(inplace=True)

In [ ]:
dat

##Deep Learning for Geospatial Data


Since there are no categorical data embedding layer will not help us here.

* ## 1.)  How you will split the data for training, validation & testing? 
-> I will be using scikit. learn train_test_split function to split the data in train and test function. If I ever need a validation dataset I will further split the dataset. I can also apply K-fold on every this dataset. 
* ## 2.)  Implementations for data loading, data transformations & inverse transformation
-> I normalized the input dataset, I saved mean and std in a dictionary to normalize in evaluation.
I will use "np. exp" to get my result. Because of np. exp is the value of np.log.
* ## 3.) Choice of Model architecture
I am using a very simple model because we don't have many features to play with here. But I added a residual connection to help in any linear relations
* ## 4.) Obtaining and Fine-tuning a pre-trained model if used
I used my own network, so I don't need to fine-tune any model.
* ## 5.) Function descriptions and definitions for model training, testing, and inference. 
-> I trained and tested in the same loop, but created a separate file that can be used to inference on it's alone.
* ## 6.)Implementation techniques that help improve the efficiency of model training and data loading. 
General methods are Normalization, Standardization, adding residual connections.
To increase the efficiency we can load our data in the GPU using .to(Cuda)
* ## 7.)How you will avoid overfitting (and implementation of solution). 
-> I added dropouts layers. And I also implemented batch norm, in some cases, it helps to apply batch norm.
* ## 8.) What do you use for visualizing model training and performance? 
I use tensorboard here. But in personal projects I also use WandB.
* ## 9.) What factors do you think might restrict the model from achieving a high accuracy?
We don't have many features to work with. Our data is too skewed. We need to apply more EDA to extract better features.

## Standardise input to the neural network

## Normalize data

In [ ]:
dat.to_csv("name.csv")

In [ ]:
dat = pd.read_csv("name.csv", index_col=None)

In [ ]:
dat

Here I am using latitude and longitude in the model because the location of data point might help in our prediction. I normalized that too becuase it gave me much better result.

In [ ]:
t2m_mean = dat["t2m"].mean()
t2m_std  = dat["t2m"].std()

swvl1_std  = dat["swvl1"].mean()
swvl1_std  = dat["swvl1"].std()

latitude_std  = dat["latitude"].mean()
latitude_std  = dat["latitude"].std()

longitude_std  = dat["longitude"].mean()
longitude_std  = dat["longitude"].std()

In [ ]:
re_normalize_vals ={
    "mean":{
        "t2m":dat["t2m"].mean(),
        "swvl1":dat["swvl1"].mean(),
        "latitude":dat["latitude"].mean(),
        "longitude": dat["longitude"].mean()
    },
    "std":{
        "t2m":dat["t2m"].std(),
        "swvl1":dat["swvl1"].std(),
        "latitude":dat["latitude"].std(),
        "longitude": dat["longitude"].std()
    }
}

In [ ]:
re_normalize_vals

## Transformation

In [ ]:
## Normalze data
dat["t2m"] = (dat["t2m"] - dat["t2m"].mean())/(1e-7  + dat["t2m"].std())
dat["swvl1"] = (dat["swvl1"] - dat["swvl1"].mean())/(1e-7  + dat["swvl1"].std())
dat["latitude"] = (dat["latitude"] - dat["latitude"].mean())/(1e-7  + dat["latitude"].std())
dat["longitude"] = (dat["longitude"] - dat["longitude"].mean())/(1e-7  + dat["longitude"].std())

In [ ]:
dat = dat.drop(["time"], axis=1)
dat

In [ ]:
Y = dat["tp"].to_numpy()
dat = dat.drop(["tp"], axis=1).to_numpy()

### Let's split the dataset. I will use 33% of data to check our values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dat, Y, test_size=0.33, random_state=4)

In [ ]:
x_train

In [ ]:
y_train

Implementations for data loading, data transformations & 

---> inverse transformation


In [ ]:
y_train

## Dataset

In [ ]:
class GeoDataset(Dataset):
  def __init__(self, X, y, transforms=None):
    self.X = X
    self.y = y
  def __len__(self):
    return len(self.y)
  def __getitem__(self, idx):
    return torch.tensor([self.X[idx][0], self.X[idx][1], self.X[idx][2], self.X[idx][3]]), torch.tensor([self.y[idx]])

In [ ]:
train_dataset = GeoDataset(x_train, y_train)

In [ ]:
test_dataset = GeoDataset(x_test, y_test)

In [ ]:
for dat in train_dataset:
  print(dat)
  break

In [ ]:
len(train_dataset)

In [ ]:
train_dl = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [ ]:
for data in train_dl:
  print(data)
  break

#### Create Model

We are going to use dropout to prevent from overfitting. Becuase we can have as much data as much we want and this problem is not that difficult our simple model can approximate it.

In [ ]:
class GeoModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.lin1 = nn.Linear(4, 4)
    self.drop = nn.Dropout(0.2)
    self.lin2 = nn.Linear(4, 8)

    self.lin3 = nn.Linear(8, 4)
    self.bn1  = nn.BatchNorm1d(4)

    self.lin4 = nn.Linear(4, 2)
    self.lin5 = nn.Linear(2, 1)

  def forward(self, X):
    x_temp = X
    x = F.relu(self.lin1(X))
    x = F.relu(self.lin2(x))
    x = self.drop(x)
    x = F.relu(self.lin3(x) + x_temp) # I added a resedual connection so if any linear pattern exists
    x = self.bn1(x)
    x = F.relu(self.lin4(x))

    x = self.lin5(x)
    return x

In [ ]:
net = GeoModel().to(dev)

In [ ]:
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(net.parameters())

In [ ]:
criterion(torch.randn(100, 128), torch.randn(100, 128)).sum()

In [ ]:
input

In [ ]:
total_loss   = 0.0
for i, data in enumerate(train_dl):
    input, label = data
    out = net(input.float().to(dev))
    print(out, label)
    break

In [ ]:
def train(train_dataloader, test_data_loader, epochs):
  for epoch in range(epochs):
    net.train()
    running_loss = 0.0
    total_loss   = 0.0
    for i, data in enumerate(train_dataloader):
      input, label = data
      optimizer.zero_grad()

      out = net(input.float().to(dev))
      loss = criterion(out, label.float().to(dev))
    #   print(out[:])
    #   print(label)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      total_loss   += loss.item()
      # break
      if i%100==99:
        writer.add_scalar('loss/train_running', running_loss/100, i)
        # tf.summary.scalar('loss', train_loss.result(), step=i)
        print(running_loss/100, epoch, i, "train")
        running_loss = 0.0

    writer.add_scalar('loss/train_total', total_loss/len(train_dataloader), epoch)
    total_loss   = 0.0
    running_loss = 0.0
    with torch.no_grad():
        net.eval()
        for i, data in enumerate(test_data_loader):
            data, labels = data
            outputs = net(data.float().to(dev))
            loss = criterion(out, label.float().to(dev))
            running_loss += loss.item()
            total_loss   += loss.item()

            # break
            if i%100==0:
              writer.add_scalar('loss/valid_running', running_loss, i)

            #   tf.summary.scalar('loss', train_loss.result(), step=i)
              print(running_loss/100, epoch, i, "val")
              running_loss = 0.0
        writer.add_scalar('loss/valid_total', total_loss/len(train_dataloader), epoch)


In [ ]:
train(train_dl, test_dl ,5)

In [ ]:
torch.save(net, "weights.pth")


# I am going to use tesnsorboard to visualize my data output.

In [ ]:
%tensorboard --logdir runs

## EVAL Method

In [ ]:
re_normalize_vals = {'mean': {'latitude': 38.40959136588079,
  'longitude': 45.36507742195719,
  'swvl1': 0.22580819576544764,
    't2m': 281.8924812033501},
    'std': {'latitude': 19.317098089735516,
  'longitude': 25.47684871130826,
  'swvl1': 0.15814413872235586,
  't2m': 16.144941173993605}}


In [ ]:
def normalizeForInput(latitude, longitude, t2m, swvl1):
    latitude = (latitude - re_normalize_vals["mean"]["latitude"])/(1e-7  + re_normalize_vals["std"]["latitude"])
    longitude = (longitude - re_normalize_vals["mean"]["longitude"])/(1e-7  + re_normalize_vals["std"]["longitude"])
    t2m = (t2m - re_normalize_vals["mean"]["t2m"])/(1e-7  + re_normalize_vals["std"]["t2m"])
    swvl1 = (swvl1 - re_normalize_vals["mean"]["swvl1"])/(1e-7  + re_normalize_vals["std"]["swvl1"])
    return latitude, longitude, t2m, swvl1

In [ ]:
net_eval = torch.load("weights.pth")

# net_eval.load_state_dict(torch.load("weights.pth")) 
net_eval.eval()

In [ ]:
out = net_eval(torch.tensor([[-85.0, 256.0, 1.334842,	0.344491]]).to(dev))

In [ ]:
def denoramlize_output(output):
    return np.exp(output)

In [ ]:
denoramlize_output(out.detach().cpu())